## ARC 380 / CEE 380 – Introduction to Robotics for Digital Fabrication
## Session 9 Code Supplement
Princeton University, Spring 2024

Professor: Arash Adel | Assistant-in-Instruction: Daniel Ruan

---

# 1 Motion Planning Algorithms

## 1.1 A* Search

### Imports and helper/visualization functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
# Grid visualization

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_grid(grid_size, start, goal, obstacles, path=None):
    fig, ax = plt.subplots(figsize=(5, 5))
    # Create a grid
    ax.set_xlim([0, grid_size[1]])
    ax.set_ylim([0, grid_size[0]])

    # Draw grid lines
    for x in range(grid_size[1] + 1):
        ax.axvline(x, lw=1, color='lightgray', zorder=0)
    for y in range(grid_size[0] + 1):
        ax.axhline(y, lw=1, color='lightgray', zorder=0)

    # Start cell
    ax.add_patch(patches.Rectangle((start[0], start[1]), 1, 1, color='green'))
    # Goal cell
    ax.add_patch(patches.Rectangle((goal[0], goal[1]), 1, 1, color='red'))
    # Obstacles
    for obs in obstacles:
        ax.add_patch(patches.Rectangle((obs[0], obs[1]), 1, 1, color='black'))
    # Path
    if path:
        for step in path:
            ax.add_patch(patches.Rectangle((step[0], step[1]), 1, 1, color='blue', alpha=0.5))

    plt.axis('off')
    plt.show()


### A* search algorithm

In [ ]:
# A* search algorithm

def a_star_search(grid_size, start, goal, obstacles):
    # Create a grid
    grid = np.zeros(grid_size, dtype=int)
    for obs in obstacles:
        grid[obs] = 1

    # Create a list to store the path
    path = []

    # Create a list to store the open and closed nodes
    open_nodes = []
    closed_nodes = []

    # Create a list to store the cost of each node
    g_cost = np.inf * np.ones(grid_size, dtype=int)
    h_cost = np.inf * np.ones(grid_size, dtype=int)

    # Add the start node to the open list
    open_nodes.append(start)
    g_cost[start] = 0
    h_cost[start] = np.abs(goal[0] - start[0]) + np.abs(goal[1] - start[1])

    # Create a list to store the neighbors
    neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while open_nodes:
        # Get the node with the lowest f_cost (g_cost + h_cost)
        current = open_nodes[0]
        for node in open_nodes:
            if g_cost[node] + h_cost[node] < g_cost[current] + h_cost[current]:
                current = node

        # Remove the current node from the open list
        open_nodes.remove(current)

        # Add the current node to the closed list
        closed_nodes.append(current)

        # Check if the current node is the goal
        if current == goal:
            # Reconstruct the path
            path.append(current)
            while current != start:
                for neighbor in neighbors:
                    next_node = (current[0] + neighbor[0], current[1] + neighbor[1])
                    if next_node in closed_nodes and g_cost[next_node] == g_cost[current] - 1:
                        path.append(next_node)
                        current = next_node
                        break
            path.reverse()
            return path

        # Expand the current node
        for neighbor in neighbors:
            next_node = (current[0] + neighbor[0], current[1] + neighbor[1])

            # Check if the next node is inside the grid
            if next_node[0] < 0 or next_node[0] >= grid_size[0] or next_node[1] < 0 or next_node[1] >= grid_size[1]:
                continue

            # Check if the next node is an obstacle
            if grid[next_node] == 1:
                continue

            # Check if the next node is in the closed list
            if next_node in closed_nodes:
                continue

            # Calculate the cost of the next node
            cost = g_cost[current] + 1

            # Check if the next node is in the open list
            if next_node not in open_nodes:
                open_nodes.append(next_node)
            elif cost >= g_cost[next_node]:
                continue

            # Update the cost of the next node
            g_cost[next_node] = cost
            h_cost[next_node] = cost + np.abs(goal[0] - next_node[0]) + np.abs(goal[1] - next_node[1])


### Test example

In [ ]:
# Grid dimensions and obstacles
grid_size = (5, 5)
obstacles = [(2, 1), (3, 1), (2, 2), (2, 3), (4, 1)]

In [ ]:
# Define start and goal
start = (0, 0)
goal = (4, 3)

# Visualize the grid
visualize_grid(grid_size, start, goal, obstacles)

In [ ]:
path = a_star_search(grid_size, start, goal, obstacles)
print(path)
visualize_grid(grid_size, start, goal, obstacles, path)

---

## 1.2 RRT Algorithm

### Imports and helper/visualization functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle


In [ ]:
def visualize_environment(obstacles, start, goal, tree):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(x_lim)
    ax.set_ylim(y_lim)

    # Draw obstacles
    for center, radius in obstacles:
        ax.add_patch(Circle(center, radius, color='red'))

    # Draw the start and goal
    ax.plot(start[0], start[1], 'go')  # start in green
    ax.plot(goal[0], goal[1], 'ro')  # goal in red

    # Draw the tree
    for node, parent_idx in tree[1:]:  # Skip the first node as it is the root and has no parent
        if parent_idx is not None:  # Safety check, though all should have parents except the root
            parent_node = tree[parent_idx][0]  # Retrieve the parent node position
            plt.plot([parent_node[0], node[0]], [parent_node[1], node[1]], 'b')

    plt.show()


In [ ]:
def distance(point1, point2):
    """Calculate the Euclidean distance between two points."""
    return np.linalg.norm(np.array(point1) - np.array(point2))

def nearest_node(tree, node):
    """Find the nearest node in the tree to the given node."""
    distances = [distance(node, n[0]) for n in tree]
    nearest_index = np.argmin(distances)
    return nearest_index

def is_collision_free(new_node, nearest_node, obstacles):
    """Check if the path between new_node and nearest_node is collision-free."""
    # Helper function to clamp a value between two limits
    def clamp(n: float, smallest: float, largest: float) -> float:
        return max(smallest, min(largest, n))

    # Helper function to calculate the distance between a point and a line segment
    def dist(edge, pt) -> float:
        edge_length = np.linalg.norm(edge[1] - edge[0])
        t = np.dot(pt - edge[0], edge[1] - edge[0]) / (edge_length ** 2)
        t = clamp(t, 0, 1)
        edge_pt = edge[0] + t * (edge[1] - edge[0])
        return np.linalg.norm(pt - edge_pt)

    edge = np.array([nearest_node, new_node])
    in_free_space = True
    for obstacle in obstacles:
        if dist(edge, obstacle[0]) <= obstacle[1]:
            in_free_space = False
            break
    return in_free_space

def step_from_to(node1, node2, step_size=1):
    """Return a point step_size away from node1 towards node2."""
    if distance(node1, node2) < step_size:
        return node2
    else:
        theta = np.arctan2(node2[1]-node1[1], node2[0]-node1[0])
        return node1[0] + step_size*np.cos(theta), node1[1] + step_size*np.sin(theta)


### RRT algorithm

In [ ]:
def rrt(start, goal, obstacles, num_iterations=500, step_size=1):
    tree = [(start_pos, None)]      # Initialize the tree with the start position and no parent
    for _ in range(num_iterations):
        # Randomly sample a point in the environment
        rand_node = (np.random.uniform(x_lim[0], x_lim[1]), np.random.uniform(y_lim[0], y_lim[1]))

        # Find the nearest node in the tree to the random node
        nearest_idx = nearest_node(tree, rand_node)

        # Step from the nearest node to the random node
        new_node = step_from_to(tree[nearest_idx][0], rand_node, step_size)

        # Check if the path between new_node and nearest_node is collision-free
        if is_collision_free(new_node, tree[nearest_idx][0], obstacles):
            tree.append((new_node, nearest_idx))

            # Check if the new node is close to the goal
            if distance(new_node, goal) <= step_size:
                print("Goal reached.")
                return tree
    print("Goal not reached.")
    return tree


### Test example

In [ ]:
# Define boundaries
x_lim = (0, 100)
y_lim = (0, 100)

# Define obstacles as circles (center and radius)
obstacles = [((40, 60), 10), ((70, 80), 15), ((20, 40), 20)]

# Start and Goal positions
start_pos = (10, 10)
goal_pos = (90, 90)

In [ ]:
num_iterations = 500
step_size = 3

tree = rrt(start_pos, goal_pos, obstacles, num_iterations, step_size)
visualize_environment(obstacles, start_pos, goal_pos, tree)